# 1) Build The Point
The result of this step will be the equipment point in rdf format. This can then be used to validate against the point SHACL shape (see step 2).

## Setup
Run the code below to setup the blank graph and shapes wrapper

In [2]:
from rdflib import Namespace

import tasty.constants as tc
import tasty.graphs as tg
import tasty.entities as te

# Create a namespace and load in a blank Brick / Haystack graph
SAMPLE = Namespace('urn:sample/')
hg = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
hg.bind('sample', SAMPLE)
h_ont = tg.load_ontology(tc.HAYSTACK, tc.V3_9_10)

# Specify the schema version (tc.V9_9_10, etc.) to use
hp = te.HaystackPointDefs(tc.V3_9_10)
he = te.HaystackEquipDefs(tc.V3_9_10)
hrefs = te.HaystackRefDefs(tc.V3_9_10)

# Bind all of the first class types as attributes
hp.bind()
he.bind()
hrefs.bind()

# Simple wrapper around all of the shapes
shrap = te.ShapesWrapper(tc.HAYSTACK, tc.V3_9_10)

shrap.bind()
shrap.bind_composite()

def print_graph(g):
    print(g.serialize(format='turtle').decode('utf-8'))

## Build Point

In [3]:
point_shape = shrap.MinimumDischargeAirFlowSetpointShape
point = point_shape.cast_to_entity()
point_id = "point-01"
point.set_namespace(SAMPLE)
point.set_id(point_id)
point.bind_to_graph(hg)

print_graph(hg)

@prefix phIoT: <https://project-haystack.org/def/phIoT/3.9.10#> .
@prefix sample: <urn:sample/> .

sample:point-01 a phIoT:discharge-air-flow-sp .




# 2) Shapes Based Validation
This step validates the output from step 1 against the predifined SHACL shapes (which are created by the tasty `generate-shapes` command)

## Setup
Run the setup below

In [4]:
import os
from pyshacl import validate
from rdflib import Graph, SH
from rdflib.util import guess_format

from tasty.shapes_loader import ShapesLoader

## Import SHACL Shapes

The ShapesLoader gives us a wrapper around loading in the shapes graphs

In [5]:
sl = ShapesLoader(tc.HAYSTACK)
shapes_graph = sl.load_all_shapes()
shapes_graph.bind('sample', SAMPLE)

## Validate

Here we validate the newly created point against the existing point SHACL shape, by making it the `targetNode` of the SHACL shape in the shapes graph.

***Items to Change***
- Change the index of `tc.PH_SHAPES_NREL` to the name of the predefined shape


In [7]:
shapes_graph.add((tc.PH_SHAPES_NREL['MinimumDischargeAirFlowSetpointShape'], SH.targetNode, point.node))
conforms, results_graph, results = validate(hg, shacl_graph=shapes_graph, ont_graph=h_ont)

conforms

False

In [11]:
print_graph(results_graph)

@prefix ph: <https://project-haystack.org/def/ph/3.9.10#> .
@prefix phCustom: <https://project-haystack.org/def/custom#> .
@prefix sample: <urn:sample/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode sample:point-01 ;
            sh:resultPath ph:hasTag ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:QualifiedMinCountConstraintComponent ;
            sh:sourceShape [ sh:path ph:hasTag ;
                    sh:qualifiedMinCount 1 ;
                    sh:qualifiedValueShape [ sh:hasValue phCustom:min ] ] ],
        [ a sh:ValidationResult ;
            sh:focusNode sample:point-01 ;
            sh:resultMessage "Less than 1 values on sample:point-01->ph:hasTag" ;
            sh:resultPath ph:hasTag ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintCompo